In [126]:
%pwd

'c:\\git\\ai_medical_agent'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\git\\ai_medical_agent'

In [127]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [128]:
def load_pdf(pdf_path):
    loader = DirectoryLoader(pdf_path, glob = "*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [129]:
extracted_data = load_pdf("data")

In [130]:
extracted_data

[Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2020-01-27T12:04:11+03:00', 'moddate': '2020-03-18T12:39:28+03:00', 'trapped': '/False', 'source': 'data\\book3.pdf', 'total_pages': 192, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2020-01-27T12:04:11+03:00', 'moddate': '2020-03-18T12:39:28+03:00', 'trapped': '/False', 'source': 'data\\book3.pdf', 'total_pages': 192, 'page': 1, 'page_label': '2'}, page_content='Andriy Burkov\nThe Hundred-Page Machine Learning Book'),
 Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2020-01-27T12:04:11+03:00', 'moddate': '2020-03-18T12:39:28+03:00', 'trapped': '/False', 'source': 'data\\book3.pdf', 'total_pages': 192, 'page': 2, 'page_label': '3'}, page_content='МАШ ИННОЕ ОБУЧЕНИЕ \nАндрей Бу

In [131]:
len(extracted_data)

192

In [132]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(documents: List[Document]) -> List[Document]:
    minimal_docs: List[Document] = []
    for doc in documents:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(page_content=doc.page_content, metadata={"source": src})
        )
    return minimal_docs

In [133]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [134]:
len(minimal_docs)

192

In [135]:
minimal_docs

[Document(metadata={'source': 'data\\book3.pdf'}, page_content=''),
 Document(metadata={'source': 'data\\book3.pdf'}, page_content='Andriy Burkov\nThe Hundred-Page Machine Learning Book'),
 Document(metadata={'source': 'data\\book3.pdf'}, page_content='МАШ ИННОЕ ОБУЧЕНИЕ \nАндрей Бурков\nбез лишних слов\n2020'),
 Document(metadata={'source': 'data\\book3.pdf'}, page_content='ББК 32.813\nУДК 004.8\nБ91 \n\t Бурков\tАндрей\nБ91   Машинное обучение без лишних слов. — СПб.: Питер, 2020. — 192 с.: ил. — (Се-\nрия «Библиотека программиста»).\n ISBN 978-5-4461-1560-0\nВсе, что вам действительно нужно знать о машинном обучении, может уместиться на паре сотен \nстраниц.\nНачнем с простой истины: машины не учатся. Типичное машинное обучение заключается в по-\nиске математической формулы, которая при применении к набору входных данных (называемых \nобучающими данными) даст желаемые результаты.\nАндрей Бурков постарался дать все необходимое, чтобы каждый мог стать отличным совре -\nменным аналитик

In [136]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=20
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [137]:
texts_chunks = text_split(minimal_docs)

In [138]:
len(texts_chunks)

796

In [139]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

In [140]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [141]:
vector = embedding.embed_query("привет")

c:\Users\romoc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [142]:
vector

[0.01188887283205986,
 0.05277395248413086,
 -0.015860583633184433,
 0.005013542715460062,
 -0.1094951331615448,
 0.03158048912882805,
 0.129938542842865,
 0.06225423514842987,
 0.0596298985183239,
 -0.010895008221268654,
 0.04514491930603981,
 0.029110023751854897,
 0.006442241836339235,
 0.014589093625545502,
 -0.020311059430241585,
 -0.07316505163908005,
 -0.010771682485938072,
 -0.0013697915710508823,
 0.004280601628124714,
 0.03440452367067337,
 -0.0382724367082119,
 -0.09565497189760208,
 0.08402137458324432,
 0.0008564451709389687,
 -0.0693507120013237,
 0.09305378794670105,
 -0.035104554146528244,
 0.03603536635637283,
 0.07073073834180832,
 0.0019175794441252947,
 0.0368557944893837,
 -0.012808534316718578,
 0.043291762471199036,
 -0.008459714241325855,
 0.02607104741036892,
 0.037902846932411194,
 -0.033274006098508835,
 -0.04586319625377655,
 0.015422811731696129,
 0.047682490199804306,
 -0.010341893881559372,
 -0.07590299099683762,
 -0.0808224231004715,
 0.06813427805900574

In [143]:
len(vector)

384

In [144]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [145]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [146]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [147]:
pc

In [148]:
# from pinecone import ServerlessSpec

# index_name = "medical_bot"

# if not pc.has_index(index_name):
#     pc.create_index(name=index_name, dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

# index = pc.Index(index_name)

from pinecone import ServerlessSpec 

index_name = "chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [149]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = texts_chunks,
    embedding=embedding,
    index_name = index_name
)

In [150]:
from langchain_pinecone import PineconeVectorStore

# это если векторный поиск уже есть и нужно добавить дополнительно чтото
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

# создается документ который будет добавлен в индекс
dswith = Document(
    page_content="В число ансамблевых методов машинного обучения входят: беггинг, стекинг, градиентный бустинг",
    metadata={"source": "книга"}
)

# добавляем в индекс
docsearch.add_documents(documents=[dswith])



['1410c7c8-0a2d-4d11-b2e8-820edf660eca']

In [151]:

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [152]:
retriever_docs = retriever.invoke("что такое субъект?")
retriever_docs

c:\Users\romoc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(id='736cae2d-39b2-407c-b1e1-1427e378d452', metadata={'source': 'data\\book3.pdf'}, page_content='присутствует только пара признаков. Это тоже может дать представление о количестве \nкластеров. Однако такой подход страдает субъективностью, подвержен ошибкам и счи-\nтается обоснованным предположением, а не научным методом.'),
 Document(id='314964f2-7143-410a-a0c8-4ae3fba7cc67', metadata={'source': 'data\\book3.pdf'}, page_content='методов. Существуют также специализированные библиотеки настроек гиперпа-'),
 Document(id='07da4f0e-1c0b-4efd-9f6e-e17097fdc5eb', metadata={'source': 'data\\book3.pdf'}, page_content='преобразования меток в числа. Например, некоторые алгоритмы требуют исполь-'),
 Document(id='ceaf012a-7634-45ef-9b72-96b0b9eaf826', metadata={'source': 'data\\book3.pdf'}, page_content='дач этого конкретного типа градиентный спуск не требуется. Однако линейная регрессия \nявляется идеальным примером для объяснения градиентного спуска.'),
 Document(id='5eca7341-2000-4203-

In [153]:
from langchain_ollama import OllamaLLM

chatModel = OllamaLLM(model="llama3.1:8b")

In [ ]:
# from langchain_ollama import OllamaLLM

# chatModel = OllamaLLM(model="llama3.1:8b")

# from langchain_openai import ChatOpenAI

# chatModel = ChatOpenAI(model = "gpt-4o")

# chatModel = ChatOpenAI(
#     model='deepseek-chat',
#     openai_api_key=""
# )

In [155]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# system_prompt = (
#     "You are an Medical assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )
system_prompt = (
    "Ты русский помощник по машинному обучению."
    "Используй следующие части извлеченного контекста, чтобы ответить "
    "на вопрос. Если ты не знаешь ответ - скажи, что ты не знаешь ответ."
    # "Используйте максимум три предложения и сохраните "
    # "ответ кратким."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [157]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [158]:
response = rag_chain.invoke({"input": "расскажи про регуляризацию, какие виды бывают"})

print(response["answer"])

c:\Users\romoc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Регуляризация - это важнейший аспект машинного обучения! Итак, давайте разберемся в этом:

Регуляризация - это техника, которая добавляет штраф к весам моделей при оптимизации. Это помогает избежать переобучения модели (overfitting) и улучшить ее генерализацию на новых неувязанных данных.

Всего существует несколько видов регуляризаций:

1. **L1-регуляризация** ( Least Absolute Shrinkage and Selection Operator): добавляет штраф в виде абсолютного значения веса. Это приводит к тому, что некоторые веса уменьшаются до нуля.
2. **L2-регуляризация** ( Ridge Regression ): добавляет штраф в виде квадрата веса. Это приводит к тому, что все веса уменьшаются, но не доходят до нуля.
3. **L1+L2-регуляризация**: сочетание L1 и L2 регуляризаций. Это дает возможность выбирать между двумя типами штрафов для разных весов.
4. **Dropout-регуляризация**: случайное отключение нейронов в сети при каждой итерации обучения. Это помогает избежать переобучения, но может привести к некоторому уменьшению точности